用 XPath 来做一个简单的爬虫，爬取某个贴吧里的所有帖子，获取每个帖子的标题，连接和帖子中图片。

In [ ]:
import requests
from lxml import etree


class TieBa:

    def __init__(self, tieba_name):
        self.tieba_name = tieba_name  # 1. 准备url，start_url
        self.start_url = "http://tieba.baidu.com/mo/q----,sz@320_240-1-3---2/m?kw={}&lp=6024".format(
            tieba_name)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Linux; Android 8.0; Pixel 2 Build/OPD3.170816.012) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Mobile Safari/537.36"}

    def parse_url(self, url):
        print(url)
        response = requests.get(url, headers=self.headers)
        return response.content

    def get_content_list(self, html_str):  # 3. 提取数据
        html = etree.HTML(html_str)
        div_list = html.xpath(
            "//body/div/div[contains(@class,'i')]")  # 返回div的列表
        content_list = []
        for div in div_list:
            item = {}
            item["href"] = "http://tieba.baidu.com/mo/q----,sz@320_240-1-3---2/" + \
                div.xpath("./a/@href")[0]
            item["title"] = div.xpath("./a/text()")[0]
            item["img_list"] = self.get_img_list(item["href"], [])
            content_list.append(item)

        # 获取下一页的url地址
        next_url = html.xpath("//a[text()='下一页']/@href")
        # 根据列表的长度，判断是否有下一页
        if len(next_url) > 0:
            next_url = "http://tieba.baidu.com/mo/q----,sz@320_240-1-3---2/" + \
                next_url[0]
        else:
            next_url = None
        return content_list, next_url

    def get_img_list(self, detail_url, img_list):  # 获取详情页的图片地址
        # 1.发送请求
        detail_html_str = self.parse_url(detail_url)
        # 2. 提取数据
        html = etree.HTML(detail_html_str)
        img_list += html.xpath("//img[@class='BDE_Image']/@src")
        # 3. 详情页翻页
        next_url = html.xpath("//a[text()='下一页']/@href")
        next_url = "http://tieba.baidu.com/mo/q----,sz@320_240-1-3---2/" + \
            next_url[0] if len(next_url) > 0 else None
        if next_url is not None:
            return self.get_img_list(next_url, img_list)
        # else不需要写
        return img_list

    def save_content_list(self, content_lsit):  # 保存数据
        for content in content_lsit:
            print(content)

    def run(self):  # 实现主要逻辑
        next_url = self.start_url
        while next_url is not None:
            # 1. 准备url，start_url
            # 2. 发送请求，获取响应
            html_str = self.parse_url(next_url)
            # 3. 提取数据
            content_list, next_url = self.get_content_list(html_str)
            # 4. 保存
            self.save_content_list(content_list)


if __name__ == '__main__':
    tieba = TieBa("传智播客")
    tieba.run()
